In [1]:
"""
To Run the Error Labeling Pipeline


"""

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

# configuring the virtual module

In [3]:
test_mode = False

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")

if not test_mode:
    time.sleep(random_sleep_sec)

print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-28 01:04:32,159 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-28 01:04:32,162 - settings - Setting database.user to celiib
INFO - 2020-11-28 01:04:32,163 - settings - Setting database.password to newceliipass
INFO - 2020-11-28 01:04:32,169 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-28 01:04:32,170 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-28 01:04:32,182 - connection - Connect

Sleeping 165 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-28 01:04:32,378 - settings - Setting enable_python_native_blobs to True


# Defining Our Table

In [5]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

In [7]:
import error_detection as ed

In [8]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class DecompositionErrorLabels(dj.Computed):
    definition="""
    -> minnie.Decomposition
    ---
    n_face_errors : int #the number of faces that were errored out
    face_idx_for_error : longblob #the face indices for the errors computed
    """
    
    
    
    key_source = (minnie.Decomposition() & "n_somas = 1" & "n_faces>500000")
                  
    def make(self,key):
        global_start = time.time()
        segment_id = key["segment_id"]
        verbose = True
        
        print(f"\n\n----- Working on {segment_id}-------")
        whole_pass_time = time.time()
        
        neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")
        
        returned_error_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,visualize_errors_at_end=False)
        
        #------- Doing the synapse Exclusion Writing ---------- #
        data_to_write_new = ed.get_error_synapse_inserts(neuron_obj,returned_error_faces,minnie=minnie,verbose=True)
        
        if len(data_to_write_new)>0:
            print("Preparing to write errored synapses")
            minnie.SynapseExclude.insert(data_to_write_new,skip_duplicates=True)
            
        #------- Doing the Label Writing ---------- #
        new_key = dict(key,
                       n_face_errors = len(returned_error_faces),
                       face_idx_for_error = returned_error_faces)
        
        
        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)
        
        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
        

In [9]:
#(schema.jobs & "table_name='__decomposition_error_labels'").delete()
#minnie.SynapseExclude.delete()
#minnie.DecompositionErrorLabels.delete()

In [10]:
import time
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 900))
print('Populate Started')
if test_mode:
    DecompositionErrorLabels.populate(reserve_jobs=True, suppress_errors=False)
else:
    DecompositionErrorLabels.populate(reserve_jobs=True, suppress_errors=True, order='random')
print('Populate Done')

print(f"Total time for DecompositionErrorLabels populate = {time.time() - start_time}")

Populate Started


INFO - 2020-11-28 01:04:33,005 - autopopulate - Found 24968 keys to populate
INFO - 2020-11-28 01:04:33,019 - connection - Transaction started
INFO - 2020-11-28 01:04:33,020 - autopopulate - Populating: {'segment_id': 864691134884740858, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}




----- Working on 864691134884740858-------


INFO - 2020-11-28 01:04:34,095 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-28 01:04:34,285 - settings - Setting enable_python_native_blobs to True


Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7f8df594b488>, <function median_mesh_center at 0x7f8df594b730>, <function n_spines at 0x7f8df594b598>, <function n_faces_branch at 0x7f8df594b2f0>, <function skeleton_distance_branch at 0x7f8df594b510>, <function spines_per_skeletal_length at 0x7f8df594b9d8>, <function no_spine_median_mesh_center at 0x7f8df594b840>]
functions_list = [<function axon_segment at 0x7f8df56ca0d0>]
Axons not keeping because of soma: {'L1': array([11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29,
       30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47,
       50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93]), 'L4': array([22, 23, 26])}

----- Working on L0 ------
-- Axon Group 0 of size 33--
   Working on soma 0, starting_node 37
Not using AIS 

KeyboardInterrupt: 